In [43]:
%env JAX_PLATFORM_NAME=cpu

import jaxtyping  # noqa: F401

%load_ext jaxtyping
# %jaxtyping.typechecker beartype.beartype

env: JAX_PLATFORM_NAME=cpu
The jaxtyping extension is already loaded. To reload it, use:
  %reload_ext jaxtyping


In [44]:
import jax
import jax.numpy as jnp
import equinox as eqx
import optax

from tqdm.notebook import trange

from jaxtyping import Array, Bool, Float

from chaogatenn.chaogate import ChaoGate
from chaogatenn.maps import (
    LogisticMap,
    DuffingMap,
    LorenzMap,
    RosslerMap,
    ChenMap,
    RosslerHyperchaosMap,
)
from chaogatenn.utils import grad_norm

In [45]:
# Training data for the AND gate
X = jnp.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=bool)  # Input combinations
AND_Y = jnp.array([0, 0, 0, 1], dtype=bool)  # AND gate output
OR_Y = jnp.array([0, 1, 1, 1], dtype=bool)  # OR gate output
XOR_Y = jnp.array([0, 1, 1, 0], dtype=bool)  # XOR
NAND_Y = jnp.array([1, 1, 1, 0], dtype=bool)  # NAND
NOR_Y = jnp.array([1, 0, 0, 0], dtype=bool)  # NOR
XNOR_Y = jnp.array([1, 0, 0, 1], dtype=bool)  # XNOR
Y = NOR_Y

In [46]:
# Map = LogisticMap(a=4.0)
# Map = LorenzMap()
# Map = DuffingMap(
#     alpha=1.0, beta=1.0, delta=0.02, gamma=8.0, omega=0.5, dt=0.01, steps=1000
# )
# Map = ChenMap(dt=1e-5, steps=100)
Map = RosslerHyperchaosMap()

In [47]:
Map(2)  # type: ignore

Array(0.04477586, dtype=float32)

In [48]:
DELTA, X0, X_THRESHOLD = jax.random.normal(jax.random.PRNGKey(42), (3,))
chao_gate = ChaoGate(DELTA=DELTA, X0=X0, X_THRESHOLD=X_THRESHOLD, Map=Map)

In [49]:
[chao_gate(x) for x in X]

[Array(0.8321855, dtype=float32),
 Array(0.83245987, dtype=float32),
 Array(0.83245987, dtype=float32),
 Array(0.83268064, dtype=float32)]

In [50]:
@eqx.filter_value_and_grad()
def compute_loss(
    chao_gate: ChaoGate, x: Bool[Array, "batch 2"], y: Bool[Array, "batch"]
) -> Float[Array, ""]:  # noqa: F821
    pred = jax.vmap(chao_gate)(x)
    # binary cross entropy
    return -jnp.mean(y * jnp.log(pred + 1e-15) + (1 - y) * jnp.log(1 - pred + 1e-15))

In [51]:
@eqx.filter_jit
def make_step(
    model: ChaoGate,
    x: Bool[Array, "dim 2"],
    y: Bool[Array, "dim"],  # noqa: F821
    optim: optax.GradientTransformation,
    opt_state: optax.OptState,
) -> (Float[Array, "dim"], ChaoGate, optax.OptState):  # type: ignore  # noqa: F821
    loss, grads = compute_loss(model, x, y)
    updates, opt_state = optim.update(grads, opt_state)
    # jax.debug.print(f"{grads, updates}")
    model = eqx.apply_updates(model, updates)
    return loss, model, opt_state

In [52]:
optim = optax.adabelief(3e-5)
opt_state = optim.init(eqx.filter(chao_gate, eqx.is_inexact_array))

In [53]:
epochs = 5_000

for epoch in trange(epochs):
    loss, chao_gate, opt_state = make_step(chao_gate, X, Y, optim, opt_state)  # type: ignore
    _, grads = compute_loss(chao_gate, X, Y)
    grad_norm_value = grad_norm(grads)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss}, Grad Norm: {grad_norm_value}")

    if loss < 1e-3:
        break

  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 0, Loss: 1.3861539363861084, Grad Norm: 0.5825067162513733
Epoch 10, Loss: 1.385933518409729, Grad Norm: 0.5824519991874695
Epoch 20, Loss: 1.3856210708618164, Grad Norm: 0.5823754668235779
Epoch 30, Loss: 1.3852266073226929, Grad Norm: 0.582279622554779
Epoch 40, Loss: 1.384760856628418, Grad Norm: 0.5821669101715088
Epoch 50, Loss: 1.3842322826385498, Grad Norm: 0.5820392370223999
Epoch 60, Loss: 1.383647084236145, Grad Norm: 0.5818980932235718
Epoch 70, Loss: 1.3830101490020752, Grad Norm: 0.5817444920539856
Epoch 80, Loss: 1.3823245763778687, Grad Norm: 0.5815792083740234
Epoch 90, Loss: 1.3815938234329224, Grad Norm: 0.5814028978347778
Epoch 100, Loss: 1.38081955909729, Grad Norm: 0.5812160968780518
Epoch 110, Loss: 1.3800036907196045, Grad Norm: 0.5810191631317139
Epoch 120, Loss: 1.379149079322815, Grad Norm: 0.5808126926422119
Epoch 130, Loss: 1.3782563209533691, Grad Norm: 0.5805966258049011
Epoch 140, Loss: 1.377326250076294, Grad Norm: 0.5803714394569397
Epoch 150, Los

In [54]:
print("\nTrained ChaoGate Parameters:")
print(
    f"DELTA: {chao_gate.DELTA}, X0: {chao_gate.X0}, X_THRESHOLD: {chao_gate.X_THRESHOLD}"
)


Trained ChaoGate Parameters:
DELTA: -1.3594744205474854, X0: -1.1463942527770996, X_THRESHOLD: 1.005868911743164


In [55]:
[
    (
        bool(x1.item()),
        bool(x2.item()),
        (
            chao_gate.Map(chao_gate.X0 + x1 * chao_gate.DELTA + x2 * chao_gate.DELTA)
            > chao_gate.X_THRESHOLD
        ).item(),
    )
    for x1, x2 in X
]

[(False, False, False),
 (False, True, False),
 (True, False, False),
 (True, True, False)]

In [56]:
pred_ys = jax.vmap(chao_gate)(X)
num_correct = jnp.sum((pred_ys > 0.5) == Y)
final_accuracy = (num_correct / len(X)).item()
print(f"final_accuracy={final_accuracy}")

final_accuracy=0.75
